---

# **Greenspace in San Jose, California by census tract**

In this analysis, I analyze greenspace in San Jose, California 

In [1]:
# Import libraries
from glob import glob
import os
import earthpy as et
import earthpy.earthexplorer as etee
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import shutil    

In [2]:
# Create project directory
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
neighborhoods_dir = os.path.join(data_dir, 'sanjose-greenspace', 'sanjose-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'sanjose-greenspace', 'ndvi')

for a_dir in [neighborhoods_dir, ndvi_dir]:
        if not os.path.exists(a_dir):
                os.makedirs(a_dir)

## Census tracts

### Select a small number of neighborhoods to test your code and loops

In the cell below, download **and cache** a shapefile of the City of Chicago neighborhoods from [the City of Chicago Data Portal](https://data.cityofchicago.org/).

To cache downloads and calculations, you will need to use a **conditional statement**, like the following example code where `condition` is some boolean value you computed:

```python
if condition:
    do_something()
```

Note that, like for `for` loops, conditionals use **indentation** to determine what happens only when the condition is `True` and what happens no matter what.

Conditional statements can also have multiple parts, although you won't need that for this first caching step:

```python
if condition1:
    do_something()
elif condition2:
    do_something_else()
else:
    do_yet_another_thing()
```

YOUR TASK:
1. IF you don't have a City of Chicago neighborhood file saved already:
   1. Download and open up the shapefile
   2. Save it to a file using the `.to_file()` method of `GeoDataFrame`s (or some other method from earlier in the semester)
2. Load in the City of Chicago dataset from a file
3. Check that your caching is working. One way to do this is to make sure that you print something to indicate when the download is happening.

In [4]:
neighborhoods_path = os.path.join(neighborhoods_dir, 'sanjose_neighborhoods.shp')
if not os.path.exists(neighborhoods_path):
    neighborhoods_url = (
   "https://geo.sanjoseca.gov/server/rest/services/OPN/OPN_OpenDataService/MapServer/549/query?outFields=*&where=1%3D1&f=geojson")
    gpd.read_file(neighborhoods_url).to_file(neighborhoods_path)

neigh_gdf = gpd.read_file(neighborhoods_path).set_index('NAME')

neigh_test_gdf = (
    neigh_gdf
    .loc[['Little Saigon and Spring Brook', 'Almaden Country Club']]
)

neigh_test_gdf

,OBJECTID,FACILITYID,INTID,SOURCE,LASTUPDATE,NOTES,ENTERPRISE,SHAPE_Leng,SHAPE_Area,geometry
NAME,,,,,,,,,,
Little Saigon and Spring Brook,65,65,65,Census Blockgroup,1653577002000,None,REF-NEIG-0000000065,21413.391954,1.913970e+07,"POLYGON ((-121.85369 37.33269, -121.85364 37.3..."
Almaden Country Club,222,222,222,None,1653577003000,None,REF-NEIG-0000000222,16402.986382,1.288214e+07,"POLYGON ((-121.85780 37.21014, -121.85776 37.2..."


In [5]:
neigh_gdf.hvplot(
    geo=True,
    line_color='black',
    fill_alpha=0,
    hover_cols=['column1', 'column2'],
    tiles='EsriImagery')

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

## Visualize greenspace

In [ ]:
# To fix Type Error
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date.

    Downloads data from the National Agricultural Imagery Program (NAIP).
    <citation>

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """

    print(f'\nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ', '-'),
        bbox=bbox,
        start=start, 
        end=end,
        store_credential=True)
    print(f'Download label: {naip_downloader.label}')
    print(f'Lower left x-coordinate: {naip_downloader.bbox.llx}')
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
  print('Reading in the NDVI statistics file...')
  ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
else:
  print('NDVI statistics file does not exist...')
  ndvi_stats_df = pd.DataFrame()

for NAME, details in neigh_test_gdf.iterrows():
    if NAME in ndvi_stats_df.index:
      print('Neighborhood stats have already been calculated. Skipping.')
      continue
    
    try:
        downloader = download_neighborhood_data(
            NAME, details.geometry, '2021-01-01', '2021-12-31')
        
    except (ValueError, TypeError):
        print('Download not available. Try different date range.')
        
        try:
            downloader = download_neighborhood_data(
                NAME, details.geometry, '2022-01-01', '2022-12-31')
        except (ValueError, TypeError):
            print('Data cannot be found. Skipping neighborhood.')
            continue

In [ ]:
help(download_neighborhood_data)

In [ ]:
def load_and_merge_arrays(name):
    """
    Load in and merge arrays

    Downloads data from the National Agricultural Imagery Program (NAIP)
    <citation>

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """

    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked=True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# # Run to function
# merged_da = load_and_merge_arrays('Almaden Country Club')
# merged_da

In [ ]:
def calculate_ndvi_statistics(gdf, da, stats_path):
    """
    Calculate NDVI, then summarize, and save statistics

    Uses downloaded National Agricultural Imagery Program (NAIP)
    data. <citation>
    

    Parameters
    ==========
    gdf : gpd.GeoDataFrame
      One row with the neighborhood name and boundary
    da:  rxr.DataArray
      Multispectral raster data
    stats_path : pathLike
      The path to the statistics file to save results.
    """
    name = str(gdf.index[0])
    print(f'Neighborhood name: {name}')

    file_is_empty = True
    if os.path.exists(stats_path):
      print('Stats file does exist.')
      stats_df = pd.read_csv(stats_path)
      with open(stats_path, 'r') as stats_file:
        file_is_empty = len(stats_df) ==0
      print(f'Stats file is empty? {file_is_empty}')

      if not file_is_empty:
        if name in stats_df.neighborhood and (not override):
          print('Stats already calculated. Skipping...')
          return

    # Reproject to same CRS
    reprojected_gdf = gdf.to_crs(da.rio.crs)

    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)

    # Calculate NDVI
    ndvi_da = (
        (da.sel(band=4) - da.sel(band=1))
        / (da.sel(band=4) + da.sel(band=1))
    )
    
    print('Writing stats to file')
    mode = "w" if file_is_empty else 'a'
    pd.DataFrame(dict(
        neighborhood=[name],
        ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
        ndvi_mean=[float(ndvi_da.mean())]
    )).to_csv(stats_path, mode=mode, header=file_is_empty, index=False)

# Run to test function
calculate_ndvi_statistics(
    neigh_test_gdf.loc[[NAME]], merged_da, ndvi_stats_path)

In [8]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

for neighborhood_name, details in neigh_test_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")

    if neighborhood_name in ndvi_stats_df.index:
        print(f'Neighborhood name: {neighborhood_name}')
        print('Neighborhood stats have already been calculated. Skipping.')
        continue

    try:
        downloader = download_neighborhood_data(
            neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
        
    except (ValueError, TypeError):
        print('Download not available. Try different date range.')
        try:
            downloader = download_neighborhood_data(
                neighborhood_name, details.geometry, '2022-01-01', '2022-12-31')
        except (ValueError, TypeError):
            print('Data cannot be found. Skipping neighborhood.')
            continue

    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        neigh_test_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)

    shutil.rmtree(downloader.data_dir)

Neighborhood name: Little Saigon and Spring Brook
Neighborhood stats have already been calculated. Skipping.
Neighborhood name: Almaden Country Club
Neighborhood stats have already been calculated. Skipping.


YOUR TASK:
1. Join your `GeoDataFrame` of Chicago neighborhoods with your NDVI statistics `DataFrame`
2. Create a Chloropleth plot using one of the statistics for the color scale
3. Write a plot headline and description.

## Wealthier and less affluent neighborhoods in San Jose, California seem to have different amounts of greenness

Little Saigon and Spring Brook, a lower income neighborhood in central San Jose, and Almaden Country Club, a wealthier neighborhood in south San Jose, appear to have different amounts of greenness as quantified with mean NDVI (Normalized Difference Vegetation Index).

In [9]:
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
chloropleth = (gv.tile_sources.EsriImagery * gv.Polygons(
    neigh_test_gdf.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_mean'])
    .opts(width=800,
          height=800,
          fill_alpha=0.4)
)
hv.save(chloropleth, 'chloropleth.html')

chloropleth

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (ndvi_mean)

In [12]:
%%capture
%%bash
jupyter nbconvert sanjose_greenspace.ipynb --to html --no-input